In [1]:
from IPython.display import Javascript
display(Javascript('IPython.notebook.execute_cells_below()'))

<IPython.core.display.Javascript object>

In [2]:
# Enable interactive plot

%matplotlib notebook
%matplotlib notebook

import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
import numpy as np

In [3]:
import math
import random
import numpy as np


def vectorToDistMatrix(coords):
    '''
    Create the distance matrix
    '''
    return np.sqrt((np.square(coords[:, np.newaxis] - coords).sum(axis=2)))


def nearestNeighbourSolution(dist_matrix):
    '''
    Computes the initial solution (nearest neighbour strategy)
    '''
    node = random.randrange(len(dist_matrix))
    result = [node]

    nodes_to_visit = list(range(len(dist_matrix)))
    nodes_to_visit.remove(node)

    while nodes_to_visit:
        nearest_node = min([(dist_matrix[node][j], j) for j in nodes_to_visit], key=lambda x: x[0])
        node = nearest_node[1]
        nodes_to_visit.remove(node)
        result.append(node)

    return result

In [4]:
import random
import numpy as np


class NodeGenerator:
    def __init__(self, width, height, nodesNumber):
        self.width = width
        self.height = height
        self.nodesNumber = nodesNumber

    def generate(self):
        xs = np.random.randint(self.width, size=self.nodesNumber)
        ys = np.random.randint(self.height, size=self.nodesNumber)

        return np.column_stack((xs, ys))

In [5]:
import math
import random
import matplotlib.pyplot as plt


class SimulatedAnnealing:
    def __init__(self, coords, temp, cooling_schedule, alpha, stopping_temp, stopping_iter):
        ''' animate the solution over time
            Parameters
            ----------
            coords: array_like
                list of coordinates
            temp: float
                initial temperature
            alpha: float
                rate at which temp decreases
            stopping_temp: float
                temerature at which annealing process terminates
            stopping_iter: int
                interation at which annealing process terminates
        '''

        self.coords = coords
        self.sample_size = len(coords)
        self.temp = temp
        self.cooling_schedule = cooling_schedule
        self.alpha = alpha
        self.stopping_temp = stopping_temp
        self.stopping_iter = stopping_iter
        self.iteration = 1

        self.dist_matrix = vectorToDistMatrix(coords)
        self.curr_solution = nearestNeighbourSolution(self.dist_matrix)
        self.best_solution = self.curr_solution

        self.solution_history = [self.curr_solution]

        self.curr_weight = self.weight(self.curr_solution)
        self.initial_weight = self.curr_weight
        self.min_weight = self.curr_weight

        self.weight_list = [self.curr_weight]

        print('Intial weight: ', self.curr_weight)

    def weight(self, sol):
        '''
        Calcuate weight
        '''
        return sum([self.dist_matrix[i, j] for i, j in zip(sol, sol[1:] + [sol[0]])])

    def acceptance_probability(self, candidate_weight):
        '''
        Acceptance probability as described in:
        https://stackoverflow.com/questions/19757551/basics-of-simulated-annealing-in-python
        '''
        return math.exp(-abs(candidate_weight - self.curr_weight) / self.temp)

    def accept(self, candidate):
        '''
        Accept with probability 1 if candidate solution is better than
        current solution, else accept with probability equal to the
        acceptance_probability()
        '''
        candidate_weight = self.weight(candidate)
        if candidate_weight < self.curr_weight:
            self.curr_weight = candidate_weight
            self.curr_solution = candidate
            if candidate_weight < self.min_weight:
                self.min_weight = candidate_weight
                self.best_solution = candidate

        else:
            if random.random() < self.acceptance_probability(candidate_weight):
                self.curr_weight = candidate_weight
                self.curr_solution = candidate

    def anneal(self):
        '''
        Annealing process with 2-opt
        described here: https://en.wikipedia.org/wiki/2-opt
        '''
        while self.temp >= self.stopping_temp and self.iteration < self.stopping_iter:
            candidate = list(self.curr_solution)
            l = random.randint(2, self.sample_size - 1)
            i = random.randint(0, self.sample_size - l)

            candidate[i: (i + l)] = reversed(candidate[i: (i + l)])

            self.accept(candidate)
            if self.cooling_schedule == 'geometric':
                self.temp *= self.alpha
            elif self.cooling_schedule == 'linear':
                self.temp -= self.alpha
            self.iteration += 1
            self.weight_list.append(self.curr_weight)
            self.solution_history.append(self.curr_solution)

        print('Minimum weight: ', self.min_weight)
        print('Improvement: ',
              round((self.initial_weight - self.min_weight) / (self.initial_weight), 4) * 100, '%')

    def animateSolutions(self):
        return self.solution_history, self.coords

    def plotLearning(self):
        plt.plot([i for i in range(len(self.weight_list))], self.weight_list)
        line_init = plt.axhline(y=self.initial_weight, color='r', linestyle='--')
        line_min = plt.axhline(y=self.min_weight, color='g', linestyle='--')
        plt.legend([line_init, line_min], ['Initial weight', 'Optimized weight'])
        plt.ylabel('Weight')
        plt.xlabel('Iteration')
        plt.show()

In [6]:
# Set the simulated annealing algorithm params'''

temp = 1000
stopping_temp = 0.00000000000001      # default = 0.00000001
cooling_schedule = 'geometric'      # Set to 'geometric' or 'linear'
alpha = 0.9998      #  default = 0.9995 for 'geometric' and  0.001 for 'linear'
stopping_iter = 20000000



'''set the dimensions of the grid'''
size_width = 200
size_height = 200

'''set the number of nodes'''
population_size = 70

'''generate the list of nodes'''
nodes = NodeGenerator(size_width, size_height, population_size).generate()


# Run simulated annealing algorithm with 2-opt'''
sa = SimulatedAnnealing(nodes, temp, cooling_schedule, alpha, stopping_temp, stopping_iter)
sa.anneal()

Intial weight:  1735.5900227173277
Minimum weight:  1393.5769161791254
Improvement:  19.71 %


In [7]:
# Plot the improvement over time

sa.plotLearning()

<IPython.core.display.Javascript object>

In [8]:
# Plot animate precalulated solutions

history, points = sa.animateSolutions()


''' animate the solution over time
    Parameters
    ----------
    hisotry : list
        history of the solutions chosen by the algorith
    points: array_like
        points with the coordinates
'''

''' approx 1500 frames for animation '''
key_frames_mult = len(history) // 1500

fig, ax = plt.subplots()

''' path is a line coming through all the nodes '''
color = ['blue', 'cyan']
line, = plt.plot([], [], color=color[0], lw=2)

def init():
    ''' initialize node dots on graph '''
    x = [points[i][0] for i in history[0]]
    y = [points[i][1] for i in history[0]]
    plt.scatter(x, y, s=25, color=color[1], marker='o')

    ''' draw axes slighty bigger  '''
    extra_x = (max(x) - min(x)) * 0.05
    extra_y = (max(y) - min(y)) * 0.05
    ax.set_xlim(min(x) - extra_x, max(x) + extra_x)
    ax.set_ylim(min(y) - extra_y, max(y) + extra_y)

    '''initialize solution to be empty '''
    line.set_data([], [])
    return line,

def update(frame):
    ''' for every frame update the solution on the graph '''
    x = [points[i, 0] for i in history[frame] + [history[frame][0]]]
    y = [points[i, 1] for i in history[frame] + [history[frame][0]]]
    line.set_data(x, y)
    return line


ani = FuncAnimation(fig, update, frames=range(0, len(history), key_frames_mult),
                    init_func=init, interval=3, repeat=False)

plt.show()

<IPython.core.display.Javascript object>

In [9]:
# Save the animation

!pip install ffmpeg

ani.save('TSP.gif', writer='pillow', fps=300)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for ffmpeg: filename=ffmpeg-1.4-py3-none-any.whl size=6084 sha256=9167d5acc2d85fbaa3ea4d1b712f0e6ca6757bd2857719d381bbe8bd89e1a706
  Stored in directory: /root/.cache/pip/wheels/64/80/6e/caa3e16deb0267c3cbfd36862058a724144e19fdb9eb03af0f
Successfully built ffmpeg
